In [6]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import seaborn as sns
import random 
import dask
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cmcrameri import cm
import babet as bb
import tqdm
import glob
import subprocess
import multiprocessing
from moarpalettes import get_palette

sns.set_theme(style="white")
sns.set_style("white")

random.seed(10)
# inidates = ['2023-10-11', '2023-10-15', '2023-10-17']
inidates = ['2023-10-15', '2023-10-17']
experiments = ['pi', 'curr', 'incr']

dask.config.set(**{'array.slicing.split_large_chunks': True})

# Import tracks

In [2]:
# Import track data
P1 = multiprocessing.Pool(40)
ifs_tracks = P1.map(bb.lagrange.Lagrange.import_medr_tracks_TE, 
                    glob.glob('/gf5/predict/AWH019_ERMIS_ATMICP/Babet/DATA/postproc/tracks/TEStitch_[!2]*'))
P1.close()

ifs_tracks = pd.concat(ifs_tracks)
ifs_tracks['date'] = pd.to_datetime(ifs_tracks.loc[:,['year','month','day','hour']])

# add era track in
era_track = pd.read_csv('/gf5/predict/AWH019_ERMIS_ATMICP/Babet/DATA/postproc/tracks/TEStitch_2023_0',
                        skipinitialspace=True)

era_track['expid'] = 'era5'
era_track['experiment'] = 'era5'
era_track['inidate'] = pd.to_datetime('2022-01-01')
era_track['number'] = 0
era_track['date'] = pd.to_datetime(era_track.loc[:,['year','month','day','hour']])

all_tracks = pd.concat([ifs_tracks,era_track])

eunice_track = era_track.query('track_id==1')

In [3]:
inidates = ['2023-10-15', '2023-10-17']
ifs_tracks_filtered = {}
ifs_babet_list = {}

# for the different inidates look for storm tracks that start around the same location to the ERA5 track at that time
for ini in inidates:
    time_filter = pd.to_datetime(ini)
    ifs_tracks_filter = ifs_tracks.groupby(['track_id','inidate','experiment','number']).agg(dict(date = min,
                                                                                                day=np.size,
                                                                                                lon = lambda x: x.iloc[0],
                                                                                                lat = lambda x: x.iloc[0],
                                                                                                msl = min))
    babet_filter = era_track[era_track['date'] > time_filter].groupby(['track_id','inidate','experiment','number']).agg(dict(date = min,
                                                                                                day=np.size,
                                                                                                lon = lambda x: x.iloc[0],
                                                                                                lat = lambda x: x.iloc[0],
                                                                                                msl = min)).iloc[0]

    # starttime criterion - must be detected within 24 hours (before or after) the ERA5 track detection time
    starttime_crit = (ifs_tracks_filter.date - babet_filter.date).dt.total_seconds().abs()<(24*60*60)
    # duration criterion - must last at least 3 days
    duration_crit = ifs_tracks_filter.day > 4  # this seems a bit arbitrary
    # starting location criterion - must start within 30 degrees of the ERA5 track starting point
    startloc_crit = np.sqrt((ifs_tracks_filter.lon - babet_filter.lon)**2+(ifs_tracks_filter.lat - babet_filter.lat)**2)<10
    # depth criterion - must reach a depth of at least 980 hPa
    minmsl_crit = ifs_tracks_filter.msl < 98000

    ifs_tracks_filtered[ini] = ifs_tracks_filter.loc[minmsl_crit&duration_crit&startloc_crit]
    ifs_babet_list[ini] = ifs_tracks.set_index(['track_id','inidate','experiment','number']).sort_index().loc[ifs_tracks_filtered[ini].index].reset_index()
ifs_babet_list = pd.concat(ifs_babet_list.values(), ignore_index=True)

In [4]:
# For members with more than one Babet-like track: Keep the one that is closest to ERA5
multi_track_members = ifs_babet_list.groupby(['inidate','experiment','number','track_id']).agg(dict(year = lambda x: 1)).groupby(['inidate','experiment','number']).count().query('year > 1')

for idx in multi_track_members.index:
    track_dists = ifs_babet_list.query('inidate=="{}" & experiment=="{}" & number=={}'.format(*idx)).groupby('track_id').apply(lambda x: bb.lagrange.Lagrange.babet_dist(x))
    num_reject = track_dists.idxmax()
    ifs_babet_list_idx = ifs_babet_list.query('inidate=="{}" & experiment=="{}" & number=={} & track_id=={}'.format(*idx,num_reject)).index
    ifs_babet_list = ifs_babet_list.drop(ifs_babet_list_idx)

In [14]:
ERA5_tracks = pd.read_hdf('/gf5/predict/AWH019_ERMIS_ATMICP/Babet/DATA/postproc/tracks/ERA5_tracks.h5')
ERA5_babet_track = ERA5_tracks.query('track_id==1')

ERA5_data = xr.open_dataset('/gf5/predict/AWH019_ERMIS_ATMICP/Babet/DATA/ERA5/EU025/sfc/2023.nc').sel(time=ERA5_babet_track.date.values)
ERA5_data['centroid_lon'] = ('time',(ERA5_babet_track.lon*4).round()/4)
ERA5_data['centroid_lat'] = ('time',(ERA5_babet_track.lat*4).round()/4)

ERA5_babet_stormframe = ERA5_data.groupby('time').apply(bb.lagrange.Lagrange.lagrangian_frame)

# 

In [15]:
ERA5_babet_stormframe

<xarray.Dataset>
Dimensions:       (storm_lon: 81, storm_lat: 81, time: 57)
Coordinates:
  * storm_lon     (storm_lon) float64 -10.0 -9.75 -9.5 -9.25 ... 9.5 9.75 10.0
  * storm_lat     (storm_lat) float64 -10.0 -9.75 -9.5 -9.25 ... 9.5 9.75 10.0
  * time          (time) datetime64[ns] 2023-10-14T03:00:00 ... 2023-10-21T21...
Data variables: (12/15)
    fg10          (time, storm_lat, storm_lon) float32 nan nan ... 12.8 12.66
    cape          (time, storm_lat, storm_lon) float32 nan nan nan ... 0.0 0.0
    mxtpr         (time, storm_lat, storm_lon) float32 nan nan nan ... 0.0 0.0
    msl           (time, storm_lat, storm_lon) float32 nan nan ... 1.017e+05
    sst           (time, storm_lat, storm_lon) float32 nan nan ... 282.0 282.1
    tcc           (time, storm_lat, storm_lon) float32 nan nan ... 0.6639 0.7346
    ...            ...
    tcwv          (time, storm_lat, storm_lon) float32 nan nan ... 6.419 6.402
    tp            (time, storm_lat, storm_lon) float32 nan nan nan ... 0.0 0.0
    v100          (time, storm_lat, storm_lon) float32 nan nan ... 0.538 0.4848
    u100          (time, storm_lat, storm_lon) float32 nan nan ... -10.73 -10.58
    centroid_lon  (time) float64 -25.0 -24.5 -23.75 -23.75 ... -2.0 -3.75 -5.0
    centroid_lat  (time) float64 38.75 39.75 40.75 41.5 ... 56.75 58.0 58.5
Attributes:
    Conventions:  CF-1.6
    history:      2024-01-10 13:52:24 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...